In [1]:
import requests
import pyodata
import petl as etl
import psycopg2 as pg

In [2]:
SERVICE_URL = "http://services.odata.org/V2/Northwind/Northwind.svc/"
conn = pg.connect("host=172.20.0.2 dbname=testcase user=airflow password=airflow")
client = pyodata.Client(SERVICE_URL, requests.Session())

In [3]:
count = client.entity_sets.Employees.get_entities().count().execute()  # Считаем кол-во записей в одата
N = 3  # Сколько строк в одной пачке
count

9

Делаем загрузку по n (в данном случае по 3) объектов. Это включает в себя выгрузки из одата n объектов и загрузка в таблицу постгрес также эти n объектов. Затем цикл повторяется

In [4]:
for i in range(0, count, N):
    employees = client.entity_sets.Employees.get_entities()
    # Мы скипаем i строк от начала, затем берем первые N строк и грузим
    employees = employees.skip(i).top(N).filter("Region eq 'WA'")
    employees = employees.select('EmployeeID,LastName,Title,TitleOfCourtesy,BirthDate,HireDate,PostalCode,ReportsTo').execute()
    
    dicts = []
    for employee in employees:
        dicts.append(employee._cache)
        
    tab1 = etl.fromdicts(dicts, header=employees[0]._cache.keys())
    etl.appenddb(table=tab1,
         dbo=conn,
         tablename ="odata_test_employee",
         schema='python',
         commit=True)

In [6]:
conn.close()